<center>
<img src="https://raw.githubusercontent.com/elaynelemos/prediction-of-orders-dmc/main/assets/img/univasf-logo.png" width=200>
<h3>
    UNIVERSIDADE FEDERAL DO VALE DO SÃO FRANCISCO
    <br>COLEGIADO DE ENGENHARIA DE COMPUTAÇÃO
</h3>

<h3>Orientador</h3>
<span>Prof. Dr. Rosalvo Ferreira de Oliveira Neto</span>

<h3>Discentes</h3>
<span>Anísio Pereira Batista Filho
<br>Edjair Aguiar Gomes Filho
<br>Elayne Rute Lessa Lemos</span>
</center>
<br><br>

## Predição de pedidos com Redes Neurais e Random Forest

Projeto em: [github.com/elaynelemos/prediction-of-orders-dmc](https://github.com/elaynelemos/prediction-of-orders-dmc)


### Importação da base de dados

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
repo_url = 'https://raw.githubusercontent.com/elaynelemos/prediction-of-orders-dmc/main'

transact_train_database = pd.read_csv(f'{repo_url}/data/transact_train.txt', sep = '|')
transact_train_database.sample(3)

In [ ]:
transact_test_database = pd.read_csv(f'{repo_url}/data/transact_class.txt', sep = '|')
transact_test_database.sample(3)

<br>

### Alteração da granularidade da base de dados

In [ ]:
# remove todas as linhas com valores de sessionNo iguais exceto a última
session_train_database = transact_train_database.drop_duplicates(subset=['sessionNo'], keep='last')

# separa variável alvo no conjunto de treinamento
session_train_X = session_train_database.iloc[:,:-1]
session_train_y = session_train_database.iloc[:,-1]
session_train_y = session_train_y.replace({'y': 1 , 'n': 0 })


# remove todas as linhas com valores de sessionNo iguais exceto a última na base de teste
session_test_X = transact_test_database.drop_duplicates(subset=['sessionNo'], keep='last')

In [ ]:
session_test_X.sample(3)

<br>

### Tratamento de valores ausentes

In [ ]:
def replace_missing_value(df, value, features):
    replaced = df[features].replace(value, np.nan)
    for column in features:
        df[column] = replaced[column]

    return df

In [ ]:
# convert_float() é baseadona solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

def convert_float(df, numeric_features):
    for column in numeric_features:
        df[column] = df[column].astype(float)

    return df

In [ ]:
def replace_missing_by_fixed_value(df, value, features):
    for column in features:
        df[column].fillna(value, inplace=True)

    return df

In [ ]:
# replace_missing_by_mean() é baseadona solução proposta no Estudo de Caso
# do livro Ciência dos Dados pelo Processo de KDD do Prof. Dr. Rosalvo Neto
# livro em: https://www.researchgate.net/publication/352749819_Ciencia_dos_Dados_pelo_Processo_de_KDD
# implementação em: https://github.com/rosalvoneto/Livro

def replace_missing_by_mean(df, numeric_features):
    for column in numeric_features:
        average = df[column].mean(axis=0)
        df[column].fillna(average, inplace=True)
    
    return df

In [ ]:
def replace_missing_by_median(df, numeric_features):
    for column in numeric_features:
        med = df[column].median(axis=0)
        df[column].fillna(med, inplace=True)
    
    return df

In [ ]:
def replace_missing_by_min(df, numeric_features):
    for column in numeric_features:
        minimum = df[column].min(axis=0)
        df[column].fillna(minimum, inplace=True)
    
    return df

In [ ]:
numeric_features = ['cMinPrice', 'cMaxPrice', 'cSumPrice', 'bMinPrice', 'bMaxPrice',
    'bSumPrice', 'bStep','maxVal', 'customerScore', 'accountLifetime', 'payments', 
    'age', 'address', 'lastOrder']

string_features = ['availability', 'onlineStatus']

customerno_dependent_feats = ['maxVal', 'customerScore', 'accountLifetime', 'payments',
    'age', 'address', 'lastOrder']

In [ ]:
# substitui '?' por NaN na base do projeto
session_train_X = replace_missing_value(session_train_X.copy(), '?', numeric_features)
session_test_X = replace_missing_value(session_test_X.copy(), '?', numeric_features)
session_train_X = replace_missing_value(session_train_X.copy(), '?', string_features)
session_test_X = replace_missing_value(session_test_X.copy(), '?', string_features)

# converte NaN para float
session_train_X = convert_float(session_train_X, numeric_features)
session_test_X = convert_float(session_test_X, numeric_features)

In [ ]:
# substitui '?' por NaN na base original
transact_train_database = replace_missing_value(transact_train_database.copy(), '?', numeric_features)
transact_train_database = replace_missing_value(transact_train_database.copy(), '?', string_features)
transact_test_database = replace_missing_value(transact_test_database.copy(), '?', numeric_features)
transact_test_database = replace_missing_value(transact_test_database.copy(), '?', string_features)

# converte NAN para float
transact_train_database = convert_float(transact_train_database, numeric_features)
transact_test_database = convert_float(transact_test_database, numeric_features)

In [ ]:
# estabelece valor fixo para valores ausentes em atributos não numéricos
session_train_X = replace_missing_by_fixed_value(session_train_X, 'ausente', string_features)
session_test_X = replace_missing_by_fixed_value(session_test_X, 'ausente', string_features)

In [ ]:
session_train_X.sample(3)

In [ ]:
session_preprocessed = {}

#### Estratégia: substituição pela média

In [ ]:
session_train_X_mean_replacing = replace_missing_by_mean(session_train_X.copy(), numeric_features)
session_test_X_mean_replacing = replace_missing_by_mean(session_test_X.copy(), numeric_features)

session_train_X_mean_replacing = replace_missing_value(session_train_X_mean_replacing.copy(),
    '?', customerno_dependent_feats)
session_test_X_mean_replacing = replace_missing_value(session_test_X_mean_replacing.copy(),
    '?', customerno_dependent_feats)

session_train_X_mean_replacing = replace_missing_by_mean(
    session_train_X_mean_replacing,
    customerno_dependent_feats
)
session_test_X_mean_replacing = replace_missing_by_mean(
    session_test_X_mean_replacing,
    customerno_dependent_feats
)

In [ ]:
# armazena estágio de pré-processamento para facilitar na exportação
session_preprocessed['session_train_X_mean_replacing'] = session_train_X_mean_replacing
session_preprocessed['session_test_X_mean_replacing'] = session_test_X_mean_replacing
session_preprocessed['session_train_y_mean_replacing'] = session_train_y.copy()

In [ ]:
session_train_X_mean_replacing.sample(3)

#### Estratégia: remoção de registros pouco relevantes para o modelo após substituição pela média

In [ ]:
# rows = session_train_database['customerNo'] == '?' 
#     and session_train_database['onlineStatus'] == 'ausente'
#     and session_train_database['availability'] == 'ausente'
temp = session_train_database[session_train_database['customerNo'] == '?']
temp = temp[temp['onlineStatus'] == '?']
temp = temp[temp['availability'] == '?']

# demonstação de que para quando customerNo, onlineStatus e availability
# são nulos, não há variabilidade na classe proporção aproximada de 1% para 'y'
temp.groupby('order').sessionNo.nunique()

In [ ]:
rows_to_drop = temp.index.values.tolist()

session_train_X_mean_drop_replacing = session_train_X_mean_replacing.drop(rows_to_drop, axis=0)
session_train_y_mean_drop_replacing = session_train_y.drop(rows_to_drop, axis=0)

In [ ]:
session_preprocessed['session_train_X_mean_drop_replacing'] = session_train_X_mean_drop_replacing
session_preprocessed['session_test_X_mean_replacing'] = session_test_X_mean_replacing.copy()
session_preprocessed['session_train_y_mean_drop_replacing'] = session_train_y_mean_drop_replacing

In [ ]:
session_train_X.sample(3)

#### Estratégia: substituição pela mediana

In [ ]:
session_train_X_median_replacing = replace_missing_by_median(session_train_X.copy(), numeric_features)
session_test_X_median_replacing = replace_missing_by_median(session_test_X.copy(), numeric_features)

In [ ]:
session_train_X_median_replacing = replace_missing_value(session_train_X_median_replacing.copy(),
    '?', customerno_dependent_feats)
session_test_X_median_replacing = replace_missing_value(session_test_X_median_replacing.copy(),
    '?', customerno_dependent_feats)

session_train_X_median_replacing = replace_missing_by_median(
    session_train_X_median_replacing,
    customerno_dependent_feats
)

session_test_X_median_replacing = replace_missing_by_median(
    session_test_X_median_replacing,
    customerno_dependent_feats
)

In [ ]:
session_preprocessed['session_train_X_median_replacing'] = session_train_X_median_replacing
session_preprocessed['session_test_X_median_replacing'] = session_test_X_median_replacing
session_preprocessed['session_train_y_median_replacing'] = session_train_y.copy()

In [ ]:
session_train_X_mean_replacing.sample(3)

#### Estratégia: substituição pelo mínimo

In [ ]:
session_train_X_min_replacing = replace_missing_by_min(session_train_X.copy(), numeric_features)
session_test_X_min_replacing = replace_missing_by_min(session_test_X.copy(), numeric_features)

In [ ]:
session_train_X_min_replacing = replace_missing_value(session_train_X_min_replacing.copy(),
    '?', customerno_dependent_feats)
session_test_X_min_replacing = replace_missing_value(session_test_X_min_replacing.copy(),
    '?', customerno_dependent_feats)

session_train_X_min_replacing = replace_missing_by_min(
    session_train_X_min_replacing,
    customerno_dependent_feats
)

session_test_X_min_replacing = replace_missing_by_min(
    session_test_X_min_replacing,
    customerno_dependent_feats
)

In [ ]:
session_preprocessed['session_train_X_min_replacing'] = session_train_X_min_replacing
session_preprocessed['session_test_X_min_replacing'] = session_test_X_min_replacing
session_preprocessed['session_train_y_min_replacing'] = session_train_y.copy()

In [ ]:
session_train_X_min_replacing.sample(3)

<br>

### Criação de variáveis

In [ ]:
# bMeanSumPriceOverTransacitions: valor médio do carrinho durante a sessão 
session_train_X['bMeanSumPriceOverTransacitions'] = transact_train_database.groupby('sessionNo').bSumPrice.mean()
session_train_X['bMeanSumPriceOverTransacitions'].fillna(0, inplace=True)
session_test_X['bMeanSumPriceOverTransacitions'] = transact_test_database.groupby('sessionNo').bSumPrice.mean()
session_test_X['bMeanSumPriceOverTransacitions'].fillna(0, inplace=True)

# meanInterationsDuration: valor médio de tempo entre uma transação e outra na sessão
session_train_X['meanInterationsDuration'] = session_train_X['duration']/transact_train_database.groupby('sessionNo').duration.count()
session_train_X['meanInterationsDuration'].fillna(0, inplace=True)
session_test_X['meanInterationsDuration'] = session_test_X['duration']/transact_test_database.groupby('sessionNo').duration.count()
session_test_X['meanInterationsDuration'].fillna(0, inplace=True)

# bMeanCountOverTransacitions: quantidade média de itens no carrinho durante a sessão 
session_train_X['bMeanCountOverTransacitions'] = transact_train_database.groupby('sessionNo').bCount.mean()
session_train_X['bMeanCountOverTransacitions'].fillna(0, inplace=True)
session_test_X['bMeanCountOverTransacitions'] = transact_test_database.groupby('sessionNo').bCount.mean()
session_test_X['bMeanCountOverTransacitions'].fillna(0, inplace=True)

In [ ]:
from re import compile as mount


regex = mount('.*_X.*')
session_X = list(filter(regex.match, list(session_preprocessed.keys())))

average_prices = transact_train_database.groupby('sessionNo').bSumPrice.mean()
average_iter_durations = session_train_X['duration']/transact_train_database.groupby('sessionNo').duration.count()
average_counts = transact_train_database.groupby('sessionNo').bCount.mean()

for key in session_X:
    # bMeanSumPriceOverTransacitions: valor médio do carrinho durante a sessão 
    session_preprocessed[key]['bMeanSumPriceOverTransacitions'] = averages.copy()
    # meanInterationsDuration: valor médio de tempo entre uma transação e outra na sessão
    session_preprocessed[key]['meanInterationsDuration'] = average_iter_durations.copy()
    # bMeanCountOverTransacitions: quantidade média de itens no carrinho durante a sessão 
    session_preprocessed[key]['bMeanCountOverTransactions'] = average_counts.copy() 

In [ ]:
columns = ['bMeanSumPriceOverTransactions', 'meanInterationsDuration', 'bMeanCountOverTransactions']


session_preprocessed['session_train_X_mean_replacing'] = replace_missing_by_mean(
    session_preprocessed['session_train_X_mean_replacing'],
    columns
)
session_preprocessed['session_test_X_mean_replacing'] = replace_missing_by_mean(
    session_preprocessed['session_test_X_mean_replacing'],
    columns
)

session_preprocessed['session_train_X_mean_drop_replacing'] = replace_missing_by_mean(
    session_preprocessed['session_train_X_mean_drop_replacing'],
    columns
)
session_preprocessed['session_test_X_mean_drop_replacing'] = replace_missing_by_mean(
    session_preprocessed['session_test_X_mean_drop_replacing'],
    columns
)

session_preprocessed['session_train_X_median_replacing'] = replace_missing_by_median(
    session_preprocessed['session_train_X_median_replacing'],
    columns
)
session_preprocessed['session_test_X_median_replacing'] = replace_missing_by_median(
    session_preprocessed['session_trest_X_median_replacing'],
    columns
)

session_preprocessed['session_train_X_min_replacing'] = replace_missing_by_min(
    session_preprocessed['session_train_X_min_replacing'],
    columns
)
session_preprocessed['session_test_X_min_replacing'] = replace_missing_by_min(
    session_preprocessed['session_test_X_min_replacing'],
    columns
)

In [ ]:
session_preprocessed['session_test_X_min_replacing'].sample(3)

<br>

### Normalização do conjunto de dados

In [ ]:
# Listagem dos X de treinamento e teste

regex = mount('.*_X.*')
session_X = list(filter(regex.match, list(session_preprocessed.keys())))

#### Ajusta Indexação e remove coluna não mais significativa

In [ ]:
for key in session_X:
    session_preprocessed[key].set_index('sessionNo', inplace=True)
    session_preprocessed[key] = session_preprocessed[key].drop(['customerNo'], axis=1)

In [ ]:
session_preprocessed['session_train_X_median_replacing'].sample(3)

#### Normalização de valores

In [ ]:
# criação das variáveis dummies

for key in session_X:
    session_preprocessed[key] = pd.get_dummies(session_preprocessed[key], prefix_sep='_')

In [ ]:
# uniformização de valores numéricos

from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()


for key in session_X:
    X_train_norm = scaler.fit_transform(session_preprocessed[key])
    X_test_norm = scaler.fit_transform(session_preprocessed[key])
    session_preprocessed[key] = pd.DataFrame(dict(zip(session_preprocessed[key].columns.values, X_train_norm.T)))

In [ ]:
session_preprocessed['session_test_X_mean_replacing'].sample(3)

<br>

### Remoção de variáveis não significativas

In [ ]:
threshold_var=0

for key in session_X:
    l_var = [x for x in session_preprocessed[key].columns if session_preprocessed[key][x].var() <= threshold_var]
    for v in l_var:
        session_preprocessed[key] = session_preprocessed[key].drop([v], axis=1)

In [ ]:
session_train_X.sample(3)

<br>

### Exportação dos dados

In [ ]:
import os


path = 'data'
if not os.path.exists(path):
    os.mkdir(path)

session = list(session_preprocessed.keys())
for key in session:
    session_preprocessed[key].to_csv(f'{path}/{key}.csv', index=False)